In [1]:
import json

# Load the JSON data
def read_json(file): #'Chinese16\Chinese16_000000000000_keypoints.json'
    with open(file, 'r') as file:
        data = json.load(file)

    # Extract part_candidates
    part_candidates = data['part_candidates'][0]

    # Print the extracted part_candidates
    # print("Part Candidates:")
    # for key, value in part_candidates.items():
        # print(f"Part {key}: {value}")
    return part_candidates

{0,  "Nose"},<br>
{1,  "Neck"},<br>
{2,  "RShoulder"},<br>
{3,  "RElbow"},<br>
{4,  "RWrist"},<br>
{5,  "LShoulder"},<br>
{6,  "LElbow"},<br>
{7,  "LWrist"},<br>
{8,  "MidHip"},<br>
{9,  "RHip"},<br>
{10, "RKnee"},<br>
{11, "RAnkle"},<br>
{12, "LHip"},<br>
{13, "LKnee"},<br>
{14, "LAnkle"},<br>
{15, "REye"},<br>
{16, "LEye"},<br>
{17, "REar"},<br>
{18, "LEar"},<br>
{19, "LBigToe"},<br>
{20, "LSmallToe"},<br>
{21, "LHeel"},<br>
{22, "RBigToe"},<br>
{23, "RSmallToe"},<br>
{24, "RHeel"},<br>
{25, "Background"}<br>
<br>
                Initial_side    Vertex          Terminal_Side<br>
0   0-1-2       Nose            Neck            RShoulder<br>
1   0-1-5       Nose            Neck            LShoulder<br>
2   1-2-3       Neck            RShoulder       RElbow<br>
3   1-5-6       Neck            LShoulder       LElbow<br>
4   2-3-4       RShoulder       RElbow          RWrist<br>
5   5-6-7       LShoulder       LElbow          LWrist<br>
6   9-2-3       RHip            RShoulder       RElbow<br>
7   12-5-6      LHip            LShoulder       LElbow<br>
8   8-9-2       MidHip          RHip            RShoulder<br>
9   8-12-5      MidHip          LHip            LShoulder<br>
10  8-9-10      MidHip          RHip            RKnee<br>
11  8-12-13     MidHip          LHip            LKnee<br>
12  9-10-11     RHip            RKnee           RAnkle<br>
13  12-13-14    LHip            LKnee           LAnkle<br>
14  10-11-22    RKnee           RAnkle          RBigToe<br>
15  13-14-19    LKnee           LAnkle          LBigToe

In [5]:
angle_indices = [
    ['0', '1', '2'],
    ['0', '1', '5'],
    ['1', '2', '3'],
    ['1', '5', '6'],
    ['2', '3', '4'],
    ['5', '6', '7'],
    ['9', '2', '3'],
    ['12', '5', '6'],
    ['8', '9', '2'],
    ['8', '12', '5'],
    ['8', '9', '10'],
    ['8', '12', '13'],
    ['9', '10', '11'],
    ['12', '13', '14'],
    ['10', '11', '22'],
    ['13', '14', '19']
]

import math
def calculate_angle(initial_side, vertex, terminal_side):
    # Calculate the vectors representing the initial side and terminal side
    vector1 = (initial_side[0] - vertex[0], initial_side[1] - vertex[1])
    vector2 = (terminal_side[0] - vertex[0], terminal_side[1] - vertex[1])
    # Calculate the dot product of the two vectors
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    # Calculate the magnitudes of the vectors
    magnitude1 = math.sqrt(vector1[0] ** 2 + vector1[1] ** 2)
    magnitude2 = math.sqrt(vector2[0] ** 2 + vector2[1] ** 2)
    # Calculate the angle between the two vectors using arctangent
    angle_rad = math.acos(dot_product / (magnitude1 * magnitude2))
    angle_deg = math.degrees(angle_rad)
    return angle_deg

def calculate_angles(part_candidates):
    angles = []
    # print(part_candidates)
    for angle_index in angle_indices:
        try:
            angle = calculate_angle(
                [part_candidates[angle_index[0]][0], part_candidates[angle_index[0]][1]],
                [part_candidates[angle_index[1]][0], part_candidates[angle_index[1]][1]],
                [part_candidates[angle_index[2]][0], part_candidates[angle_index[2]][1]]
            )
            angles.append(angle)
        # in case openpose didn't calculate/didn't find a given keypoint
        except Exception as e:
            # print(f"Error calculating angle: {e}")
            if len(angles) > 0:
                angles.append(angles[-1])  # Append the previous row's data
                # could lead to false positive for poses later on, but easiest to implement right now.
                    # Better solution would be to average valid keypoints before and after this, 
                    # but gets computationally complex if there are consecutive invalid poses
            else:
                # Handle the case where there are no previous rows to append
                angles.append(-1)

    
    midhip_x = part_candidates["8"][0]
    midhip_y = part_candidates["8"][1]
    angles.append(midhip_x)
    angles.append(midhip_y)
    return angles

In [6]:
import os
import pandas as pd
directory = "Chinese16"

# midhip_x = []
# midhip_y = []
in_dir = "../data/1_openpose_points"
out_dir = "../data/2_angles"
for subdir in os.listdir(in_dir):
    subdir_path = os.path.join(in_dir, subdir)
    df = pd.DataFrame(
        columns= [
            "Nose-Neck-RShoulder",
            "Nose-Neck-LShoulder",
            "Neck-RShoulder-RElbow",
            "Neck-LShoulder-LElbow",
            "RShoulder-RElbow-RWrist",
            "LShoulder-LElbow-LWrist",
            "RHip-RShoulder-RElbow",
            "LHip-LShoulder-LElbow",
            "MidHip-RHip-RShoulder",
            "MidHip-LHip-LShoulder",
            "MidHip-RHip-RKnee",
            "MidHip-LHip-LKnee",
            "RHip-RKnee-RAnkle",
            "LHip-LKnee-LAnkle",
            "RKnee-RAnkle-RBigToe",
            "LKnee-LAnkle-LBigToe",
            "MidHip-X",
            "MidHip-Y"
        ]
    )
    for filename in os.listdir(subdir_path):
        filepath = os.path.join(subdir_path, filename)
        # print(calculate_angles(read_json(filepath)))
        parts_candidates = read_json(filepath)
        # print(calculate_angles(parts_candidates))
        df.loc[len(df)] = calculate_angles(parts_candidates)

# read_json(filepath)
    output_path = f"{out_dir}/{subdir}.csv"
    df.to_csv(output_path)